In [1]:
import geopandas as gpd
import shapely
import rtree
import sqlalchemy

print("--- DIAGNÓSTICO DE ENTORNO ---")
print(f"GeoPandas versión: {gpd.__version__}")
print(f"Shapely versión: {shapely.__version__}")
print(f"Rtree (Índice Espacial): {'INSTALADO' if rtree else 'NO DETECTADO'}")
print(f"SQLAlchemy versión: {sqlalchemy.__version__}")

# Prueba de creación de geometría
try:
    puntos = gpd.GeoSeries.from_xy([1, 2], [3, 4])
    print("Creación de geometrías: OK")
except Exception as e:
    print(f"ERROR creando geometrías: {e}")

--- DIAGNÓSTICO DE ENTORNO ---
GeoPandas versión: 1.1.2
Shapely versión: 2.1.2
Rtree (Índice Espacial): INSTALADO
SQLAlchemy versión: 2.0.45
Creación de geometrías: OK


In [ ]:
import geopandas as gpd
import os

# Ajusta estas rutas al nombre real de tus archivos .shp
rutas_shp = {
    "Atlas_Riesgo": "atlas_de_riesgo_inundaciones/atlas_de_riesgo_inundaciones.shp",
    "Encharcamientos": "Presencia de encharcamientos del año 2000 al 2017 en la Ciudad de México/encharcamientos_2000_2017_e.shp"
}

def auditar_shp(nombre, ruta):
    print(f"\n{'='*40}")
    print(f"AUDITORÍA: {nombre}")
    print(f"{'='*40}")
    
    # Buscar archivo .shp si el nombre no es exacto
    if not os.path.exists(ruta):
        directorio = os.path.dirname(ruta)
        archivos = [f for f in os.listdir(directorio) if f.endswith('.shp')]
        if archivos:
            ruta = os.path.join(directorio, archivos[0])
            print(f"Ruta ajustada automáticamente a: {ruta}")
        else:
            print(f"ERROR: No se encontró ningún .shp en {directorio}")
            return

    try:
        gdf = gpd.read_file(ruta)
        
        print(f"1. Sistema de Coordenadas (CRS): {gdf.crs}")
        print(f"2. Tipo de Geometría: {gdf.geom_type.unique()}")
        print(f"3. Total de Polígonos/Puntos: {len(gdf)}")
        print(f"4. Columnas disponibles:")
        print(list(gdf.columns))
        print(f"5. Muestra de datos (Primeras 3 filas):")
        # Mostrar columnas relevantes (evitando la geometría para no saturar)
        cols_visual = [c for c in gdf.columns if c != 'geometry'][:5] 
        print(gdf[cols_visual].head(3))
        
    except Exception as e:
        print(f"Error leyendo el SHP: {e}")

# Ejecutar auditoría
# Asumiendo estructura de carpetas basada en tu imagen
base_dir = "." # O la ruta donde tengas tus carpetas
rutas_reales = {
    "Atlas_Riesgo": os.path.join(base_dir, "atlas_de_riesgo_inundaciones", "atlas_de_riesgo_inundaciones.shp"),
    "Encharcamientos": os.path.join(base_dir, "Presencia de encharcamientos del año 2000 al 2017 en la Ciudad de México", "encharcamientos_2000_2017_e.shp")
}

for k, v in rutas_reales.items():
    auditar_shp(k, v)


AUDITORÍA: Atlas_Riesgo
1. Sistema de Coordenadas (CRS): EPSG:4326
2. Tipo de Geometría: ['Polygon']
3. Total de Polígonos/Puntos: 4908
4. Columnas disponibles:
['_id', 'id', 'g_pnt_2', 'geo_shp', 'fenomen', 'taxonom', 'r_p_v_e', 'intnsdd', 'descrpc', 'fuente', 'cvegeo', 'alcaldi', 'entidad', 'area_m2', 'perim_m', 'perd_rt', 'intens_n', 'intns_nm', 'int2', 'geometry']
5. Muestra de datos (Primeras 3 filas):
   _id  id                       g_pnt_2  \
0    1  35  19.2422518066,-99.0731823535   
1    2  56  19.2236128042,-99.1083418839   
2    3  32  19.3809650541,-99.1373113251   

                                             geo_shp             fenomen  
0  {"type": "Polygon", "coordinates": [[[-99.0760...  Hidrometeorolgicos  
1  {"type": "Polygon", "coordinates": [[[-99.1064...  Hidrometeorolgicos  
2  {"type": "Polygon", "coordinates": [[[-99.1366...  Hidrometeorolgicos  

AUDITORÍA: Encharcamientos
1. Sistema de Coordenadas (CRS): EPSG:32614
2. Tipo de Geometría: ['Point' None]
3.

In [6]:
import pandas as pd
import geopandas as gpd
import os

# --- CONFIGURACIÓN DE RUTAS ---
# Ajusta si tus rutas cambiaron
ruta_atlas = r"atlas_de_riesgo_inundaciones/atlas_de_riesgo_inundaciones.shp"
ruta_encharcamientos = r"Presencia de encharcamientos del año 2000 al 2017 en la Ciudad de México/encharcamientos_2000_2017_e.shp"

rutas_csv = {
    "Hechos": r"Data_Tratado\Hechos_Transito.csv",
    "Inviales": r"Data_Tratado\Inviales.csv",
    "Metro": r"Data_Tratado\Afluencia_Metro.csv"
}

# A) ATLAS DE RIESGO (Polígonos)
gdf_atlas = gpd.read_file(ruta_atlas)
# Mapeo de riesgo a valor numérico (1, 2, 3)
# Asumimos que 'intens_n' contiene texto como 'Muy Alto', 'Alto', 'Medio', 'Bajo'
# Si los valores son diferentes, el fillna lo dejará en 1 (Bajo) por defecto
mapa_riesgo = {'Muy Alto': 3, 'Alto': 3, 'Medio': 2, 'Bajo': 1, 'Muy Bajo': 1}
gdf_atlas['SCORE_ATLAS'] = gdf_atlas['intens_n'].map(mapa_riesgo).fillna(1).astype(int)
gdf_atlas = gdf_atlas[['geometry', 'SCORE_ATLAS', 'fenomen']] # Solo columnas útiles

# B) ENCHARCAMIENTOS HISTÓRICOS (Puntos -> Buffers)
gdf_ench = gpd.read_file(ruta_encharcamientos)
gdf_ench = gdf_ench.dropna(subset=['geometry']) # Eliminar geometrías nulas

# Nota: Para hacer buffer en metros, mantenemos EPSG:32614 temporalmente
print("   - Generando buffers de 50m para encharcamientos históricos...")
gdf_ench['geometry'] = gdf_ench.geometry.buffer(50) # 50 metros de radio
# Reproyectar a EPSG:4326 para coincidir con el resto
gdf_ench = gdf_ench.to_crs(epsg=4326)
gdf_ench['FLAG_HISTORICO'] = 1 # Bandera binaria
gdf_ench = gdf_ench[['geometry', 'FLAG_HISTORICO', 'TIRANTE']]

def enriquecer_dataset(ruta_csv, lat_col, lon_col, nombre_salida):
    if not os.path.exists(ruta_csv):
        print(f"Archivo no encontrado: {ruta_csv}")
        return

    print(f"\nProcesando: {nombre_salida}...")
    df = pd.read_csv(ruta_csv)
    
    # Validar que existan coords
    if lat_col not in df.columns or lon_col not in df.columns:
        print(f"   ⚠ Columnas {lat_col}/{lon_col} no encontradas. Saltando.")
        return

    # Convertir a GeoDataFrame (Puntos)
    gdf_puntos = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df[lon_col], df[lat_col]), crs="EPSG:4326"
    )

    # A) Spatial Join con ATLAS (Punto dentro de Polígono)
    # op='within' o 'intersects'
    gdf_join_1 = gpd.sjoin(gdf_puntos, gdf_atlas, how="left", predicate="intersects")
    # Limpieza post-join (eliminar index_right generado)
    gdf_join_1.drop(columns=['index_right'], errors='ignore', inplace=True)
    
    # Rellenar nulos (Si no cae en polígono, riesgo = 0 o 1)
    gdf_join_1['SCORE_ATLAS'] = gdf_join_1['SCORE_ATLAS'].fillna(0).astype(int)

    # B) Spatial Join con ENCHARCAMIENTOS (Punto dentro de Buffer Histórico)
    gdf_join_2 = gpd.sjoin(gdf_join_1, gdf_ench, how="left", predicate="intersects")
    gdf_join_2.drop(columns=['index_right'], errors='ignore', inplace=True)
    
    # Rellenar nulos
    gdf_join_2['FLAG_HISTORICO'] = gdf_join_2['FLAG_HISTORICO'].fillna(0).astype(int)
    
    # C) Cálculo de Semáforo Base (Estático)
    # Fórmula preliminar: Riesgo Atlas (0-3) + (Histórico * 2). Escala final aprox 0 a 5.
    gdf_join_2['NIVEL_RIESGO_ESTATICO'] = gdf_join_2['SCORE_ATLAS'] + (gdf_join_2['FLAG_HISTORICO'] * 2)

    # Guardar Resultado
    ruta_final = f"Data_Tratado/{nombre_salida}_Enriquecido.csv"
    
    # Eliminamos la columna geometry para guardar como CSV plano
    df_final = pd.DataFrame(gdf_join_2.drop(columns='geometry'))
    df_final.to_csv(ruta_final, index=False, encoding='utf-8-sig')
    print(f"Guardado: {ruta_final} | Registros: {len(df_final)}")
    print(f"Distribución Riesgo: {df_final['NIVEL_RIESGO_ESTATICO'].value_counts().to_dict()}")

enriquecer_dataset(rutas_csv["Hechos"], 'latitud', 'longitud', "Hechos_Transito")
enriquecer_dataset(rutas_csv["Inviales"], 'latitud', 'longitud', "Inviales")
enriquecer_dataset(rutas_csv["Metro"], 'LATITUD', 'LONGITUD', "Afluencia_Metro")

   - Generando buffers de 50m para encharcamientos históricos...

Procesando: Hechos_Transito...
Guardado: Data_Tratado/Hechos_Transito_Enriquecido.csv | Registros: 132065
Distribución Riesgo: {3: 92564, 1: 39466, 0: 19, 2: 16}

Procesando: Inviales...
Guardado: Data_Tratado/Inviales_Enriquecido.csv | Registros: 11559010
Distribución Riesgo: {3: 9005248, 1: 2533454, 2: 18001, 0: 2307}

Procesando: Afluencia_Metro...
Guardado: Data_Tratado/Afluencia_Metro_Enriquecido.csv | Registros: 5139574
Distribución Riesgo: {3: 3718892, 1: 1357410, 0: 63272}
